In [1]:
#r "nuget: FsHttp"

open FsHttp
open FSharp.Data

type Currency = Currency of int
let playerCostToUpkeep c =
    if c < 56000 then (Currency 10000)
    elif c < 106000 then (Currency 20000)
    elif c < 156000 then (Currency 30000)
    elif c < 206000 then (Currency 40000)
    elif c < 301000 then (Currency 50000)
    else (Currency 60000)

type JsonPosition = JsonProvider<"https://fumbbl.com/api/position/get/36290">
type Position =
    | Shaman
    | PlayerCoach
    | Cheerleader
    | Filcher
    | BribedRef
    | Fan
    | SquigTamer

let getPosition (i:int) = 
    http {
        GET $"https://fumbbl.com/api/position/get/{i}"
    }
    |> toText
    |> JsonPosition.Parse

let toPosition p = 
    match p with
    | "Shaman" -> Some Shaman
    | "Player-Coach" -> Some PlayerCoach
    | "Cheerleader" -> Some Cheerleader
    | "Filcher" -> Some Filcher
    | "Bribed Ref" -> Some BribedRef
    | "Fan" -> Some Fan
    | "Squig Tamer" -> Some SquigTamer
    | _ -> None

let toBowlbot p =
    match p with
    | Shaman -> "bb1d6,1d8"
    | PlayerCoach -> "bb1dX,1d15"
    | Cheerleader -> "bb1d6"
    | Filcher -> "bb1d6"
    | BribedRef -> "bb1d6"
    | Fan -> "bb1d6"
    | SquigTamer -> "bb1d6"

type Player = {
    Name : string
    Cost : Currency option
    Position : Position option
}

type Player2 = 
    | Normal of string
    | Star of string * Currency
    | Special of string * string

type FanFactor = FanFactor of int

type JsonTeam = JsonProvider<"https://fumbbl.com/api/team/get/997349">
type Team = {
    Name : string
    Treasury : Currency
    FanFactor : FanFactor
    Players : Player list
}

let getTeam (i:int) =
    http {
        GET $"https://fumbbl.com/api/team/get/{i}"
    }
    |> toText
    |> JsonTeam.Parse
    |> fun t -> {
            Name = t.Name
            Treasury = (Currency t.Treasury)
            FanFactor = (FanFactor t.FanFactor)
            Players =
                t.Players 
                |> Array.toList
                |> List.map (fun p -> 
                    { 
                        Name = p.Name; 
                        Cost = 
                            if p.Position.Equals(p.Name) then 
                                getPosition p.PositionId
                                |> fun pos -> Some (playerCostToUpkeep pos.Cost)
                            else 
                                None;
                        Position = toPosition p.Position
                    })
        }

type JsonMatch = JsonProvider<"https://fumbbl.com/api/match/get/4284928">
type MatchResult = {
    Winnings : Currency option
}

type Match = {
    Team1 : Team * MatchResult
    Team2 : Team * MatchResult
}

let getMatch (i:int) = 
    http {
        GET $"https://fumbbl.com/api/match/get/{i}"
    }
    |> toText
    |> JsonMatch.Parse
    |> fun m -> { 
            Team1 = ((getTeam m.Team1.Id), {Winnings = if m.Team1.Winnings > 0 then (Some (Currency m.Team1.Winnings)) else None})
            Team2 = ((getTeam m.Team2.Id), {Winnings = if m.Team2.Winnings > 0 then (Some (Currency m.Team2.Winnings)) else None})
        }

let toMatchCard (t,r) =
    let teamname = t.Name
    let winnings = 
        match r.Winnings with
        | Some (Currency w) -> w
        | None -> 0 
    let (Currency treasury) = t.Treasury
    let upkeep = 
        t.Players 
        |> List.map (fun p -> p.Cost) 
        |> List.fold 
            (fun acc elem -> 
                let cost = 
                    match elem with
                    | Some (Currency c) -> c
                    | _ -> 0
                acc + cost) 0
    let countPosition pos =
        t.Players
            |> List.filter (fun p -> p.Position = Some pos)
            |> List.length

    let roll pos =
        if countPosition pos > 0 
        then toBowlbot pos
        else "Not rostered"
        
    $"{teamname}\n" +
    $"XXXLBowl #1\n" +
    $"Treasury: {treasury-winnings}\n"+
    $":winnings: {winnings}\n" +
    $":ref: {roll BribedRef}\n" +
    $":fan: {roll Fan}\n" +
    $":upkeep: {upkeep}\n" +
    $"Future Treasury: {treasury-upkeep}\n" +
    $"Specialist Rolls:\n" +
    $":cheer: {roll Cheerleader}\n" +
    $":fan: {roll Fan}\n" +
    $":filcher: {roll Filcher}\n" +
    $":coach: {roll PlayerCoach}\n" +
    $":shaman: {roll Shaman}\n" +
    $":tamer: {roll SquigTamer}\n"

let myMatch = getMatch 4284928
toMatchCard myMatch.Team2



Black Tooth Rips
XXXLBowl #1
Treasury: 11000
:winnings: 70000
:ref: bb1d6
:fan: bb1d6
:upkeep: 70000
Future Treasury: 11000
Specialist Rolls:
:cheer: bb1d6
:fan: bb1d6
:filcher: bb1d6
:coach: Not rostered
:shaman: bb1d6,1d8
:tamer: Not rostered
